# Machine learning model LaQuiniela

In this notebook we are going to develop a machine learning model to predict, given the season, the division and the matchday, the result of each match, given the season, the division and the matchday.

The library `sqlite3` is for reading the files `.sqlite`, in particular, `laliga.sqlite`, with the information of the matches, and `clasification.sqlite`, where we have store the data from exercise 10.

The library `sklearn` is for machine learning. 

In [91]:
import pandas as pd
import numpy as np
import sqlite3

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

Leo los datos usando la libreia `sqlite3`.

In [92]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 
df

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
48775,2021-2022,2,42,5/29/22,None,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,5/29/22,None,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,5/29/22,None,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,5/29/22,None,CD Tenerife,FC Cartagena,None


In [93]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close() 
df_teams

,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2021-2022,1,1.0,Real Madrid,1,4.0,1.0,3.0,1.0,0.0,0.0,3.0
1,2021-2022,1,2.0,Sevilla FC,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
2,2021-2022,1,3.0,Barcelona,1,4.0,2.0,2.0,1.0,0.0,0.0,3.0
3,2021-2022,1,4.0,Atlético Madrid,1,2.0,1.0,1.0,1.0,0.0,0.0,3.0
4,2021-2022,1,5.0,Valencia,1,1.0,0.0,1.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
97549,1928-1929,1,6.0,Athletic Madrid,18,43.0,41.0,2.0,8.0,8.0,2.0,26.0
97550,1928-1929,1,7.0,Espanyol,18,32.0,38.0,-6.0,7.0,7.0,4.0,25.0
97551,1928-1929,1,8.0,Catalunya,18,45.0,49.0,-4.0,6.0,8.0,4.0,22.0
97552,1928-1929,1,9.0,Real Unión,18,40.0,42.0,-2.0,5.0,11.0,2.0,17.0


We add a new column to `df_games`, with the result of each match: (1) home team, (X) tie or (2) visitor. If the game hasn't been played yet: `None`. 

Also, we have eliminate the columns that we consider irrelevant for or model: `date`, `time`, `score`.

In [94]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = '1'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'X'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[1]), "result"] = '2'

df_games.drop(["date", "time", "score"], axis=1, inplace=True)
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,2
1,1928-1929,1,1,Espanyol,Real Unión,1
2,1928-1929,1,1,Real Madrid,Catalunya,1
3,1928-1929,1,1,Donostia,Athletic,X
4,1928-1929,1,1,Racing,Barcelona,2
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


Now we split it in two data frames, one for the `home_team` an another for the `away_team`.

In [95]:
df_games_home = df_games[["season", "division", "matchday", "home_team", "result"]]
df_games_visitor = df_games[["season", "division", "matchday", "away_team", "result"]]

The following variables are the ones provided by the user.

In [96]:
# info for the training
season_train = "1990:2000"
# info for predict
division = 1
season = "2019-2020"
matchday = 1

We prepare the dataframes to train the model. First we select only the season from the variable `season_train`. In this example, `2000:2010` means from season `2000-2001` to `2009-2010`.

In [97]:
# Data frame of clasification
df_teams_train = df_teams

df_teams_train["season"] = df_teams_train["season"].astype(str)
df_teams_train = df_teams_train.loc[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]


In [98]:
df_games_home_train = df_games_home

df_games_home_train.loc[:, "season"]= df_games_home_train["season"].astype(str)

df_games_home_train = df_games_home_train.loc[(df_games_home_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_home_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]

In [99]:
df_games_visitor_train = df_games_visitor

df_games_visitor_train.loc[:,"season"] = df_games_visitor_train["season"].astype(str)

df_games_visitor_train = df_games_visitor_train[(df_games_visitor_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_visitor_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]

In [100]:
#función merge and clean
def merge_and_clean_home(df_games, df_teams):
    # Copiar el DataFrame para evitar modificar el original
    df_train = pd.merge(df_games.copy(), df_teams.copy(), how="outer", 
                             left_on=['home_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"])

    # Eliminar filas con valores nulos en la columna 'home_team'
    df_train.dropna(subset=["home_team"], inplace=True)

    return df_train

In [101]:
#función merge and clean
def merge_and_clean_visitor(df_games, df_teams):
    # Copiar el DataFrame para evitar modificar el original
    df_train = pd.merge(df_games.copy(), df_teams.copy(), how="outer", 
                             left_on=['away_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"])

    # Eliminar filas con valores nulos en la columna 'home_team'
    df_train.dropna(subset=["away_team"], inplace=True)

    return df_train

In [102]:
df_train_home = merge_and_clean_home(df_games_home_train, df_teams_train)
df_train_away = merge_and_clean_visitor(df_games_visitor_train, df_teams_train)

In [103]:
df_train = pd.concat([df_train_home, df_train_away])
df_train

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,1990-1991,1,1,Espanyol,2,19.0,Espanyol,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,NaN
1,1990-1991,1,1,CD Logroñés,2,15.0,CD Logroñés,1.0,2.0,-1.0,0.0,1.0,0.0,0.0,NaN
2,1990-1991,1,1,Real Madrid,1,5.0,Real Madrid,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
3,1990-1991,1,1,Real Valladolid,X,11.0,Real Valladolid,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
4,1990-1991,1,1,Real Betis,X,7.0,Real Betis,2.0,2.0,0.0,0.0,0.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005,1999-2000,2,42,NaN,1,3.0,Villarreal,61.0,46.0,15.0,18.0,12.0,12.0,66.0,Villarreal
8006,1999-2000,2,42,NaN,1,7.0,Levante,55.0,52.0,3.0,16.0,13.0,13.0,61.0,Levante
8007,1999-2000,2,42,NaN,X,6.0,CP Mérida,41.0,34.0,7.0,16.0,11.0,15.0,63.0,CP Mérida
8008,1999-2000,2,42,NaN,2,19.0,Getafe,39.0,51.0,-12.0,13.0,20.0,9.0,48.0,Getafe


In [104]:
le = LabelEncoder()
df_train['team'] = le.fit_transform(df_train['team'])

In [105]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['team', 'rank', 'GD', "W", "Pts"]]  
y_train = df_train['result']  

In [106]:
def select_data(df):
    df_test = df[(df["season"] == season) & 
                         (df["division"] == division) & 
                         (df["matchday"] == matchday)]
    return df_test

In [107]:
df_teams_test = select_data(df_teams)
df_games_home_test = select_data(df_games_home)
df_games_visitor_test = select_data(df_games_visitor)

In [110]:
df_games_home

,season,division,matchday,home_team,result
0,1928-1929,1,1,Arenas Club,2
1,1928-1929,1,1,Espanyol,1
2,1928-1929,1,1,Real Madrid,1
3,1928-1929,1,1,Donostia,X
4,1928-1929,1,1,Racing,2
...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,None
48776,2021-2022,2,42,Real Sociedad B,None
48777,2021-2022,2,42,Sporting Gijón,None
48778,2021-2022,2,42,CD Tenerife,None


In [108]:
df_test_home = merge_and_clean_home(df_games_home_test, df_teams_test)
df_test_away = merge_and_clean_visitor(df_games_visitor_test, df_teams_test)

KeyError: 'home_team'

In [ ]:
df_test = pd.concat([df_test_home, df_test_away])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2019-2020,1,1,Athletic,1,6.0,Athletic,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
1,2019-2020,1,1,Celta de Vigo,2,19.0,Celta de Vigo,1.0,3.0,-2.0,0.0,1.0,0.0,0.0,NaN
2,2019-2020,1,1,Valencia,X,12.0,Valencia,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
3,2019-2020,1,1,RCD Mallorca,1,3.0,RCD Mallorca,2.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN
4,2019-2020,1,1,CD Leganés,2,16.0,CD Leganés,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,NaN
5,2019-2020,1,1,Villarreal,X,10.0,Villarreal,4.0,4.0,0.0,0.0,0.0,1.0,1.0,NaN
6,2019-2020,1,1,Alavés,1,5.0,Alavés,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
7,2019-2020,1,1,Espanyol,2,20.0,Espanyol,0.0,2.0,-2.0,0.0,1.0,0.0,0.0,NaN
8,2019-2020,1,1,Real Betis,2,13.0,Real Betis,1.0,2.0,-1.0,0.0,1.0,0.0,0.0,NaN
9,2019-2020,1,1,Atlético Madrid,1,7.0,Atlético Madrid,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN


In [ ]:
le = LabelEncoder()
df_test['team'] = le.fit_transform(df_test['team'])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2019-2020,1,1,Athletic,1,6.0,1,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
1,2019-2020,1,1,Celta de Vigo,2,19.0,6,1.0,3.0,-2.0,0.0,1.0,0.0,0.0,NaN
2,2019-2020,1,1,Valencia,X,12.0,18,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
3,2019-2020,1,1,RCD Mallorca,1,3.0,11,2.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN
4,2019-2020,1,1,CD Leganés,2,16.0,5,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,NaN
5,2019-2020,1,1,Villarreal,X,10.0,19,4.0,4.0,0.0,0.0,0.0,1.0,1.0,NaN
6,2019-2020,1,1,Alavés,1,5.0,0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
7,2019-2020,1,1,Espanyol,2,20.0,7,0.0,2.0,-2.0,0.0,1.0,0.0,0.0,NaN
8,2019-2020,1,1,Real Betis,2,13.0,12,1.0,2.0,-1.0,0.0,1.0,0.0,0.0,NaN
9,2019-2020,1,1,Atlético Madrid,1,7.0,2,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN


In [ ]:
X_test = df_test[['team','rank', 'GD', "W", "Pts"]] 
y_test = df_test['result']

In [ ]:
# Crear el clasificador RandomForest
rf_classifier = RandomForestClassifier()

# Definir los parámetros que deseas ajustar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda de cuadrícula en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", grid_search.best_params_)

# Obtener el modelo con los mejores parámetros
best_rf_model = grid_search.best_estimator_

Mejores parámetros: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}


In [ ]:
# Evaluar el modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)
y_pred

array(['1', '2', 'X', '1', '1', 'X', '1', '1', '1', '1', '1', '1', 'X',
       '1', '1', 'X', '1', '1', '1', '1'], dtype=object)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
predictions = best_rf_model.predict_proba(X_test)

print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix:\n{confusion_mat}\n")
print(f"Classification Report:\n{class_report}\n")
print(f"Predictions probability:\n{predictions}\n")

Accuracy: 0.65

Confusion Matrix:
[[8 0 0]
 [7 1 0]
 [0 0 4]]

Classification Report:
              precision    recall  f1-score   support

           1       0.53      1.00      0.70         8
           2       1.00      0.12      0.22         8
           X       1.00      1.00      1.00         4

    accuracy                           0.65        20
   macro avg       0.84      0.71      0.64        20
weighted avg       0.81      0.65      0.57        20


Predictions probability:
[[9.00517192e-01 9.61494744e-02 3.33333333e-03]
 [4.51832614e-01 5.46402680e-01 1.76470588e-03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [7.60805128e-01 2.36902172e-01 2.29269989e-03]
 [5.81885158e-01 4.06742293e-01 1.13725490e-02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [9.21514154e-01 7.71760525e-02 1.30979391e-03]
 [6.43960063e-01 3.54275231e-01 1.76470588e-03]
 [7.25456736e-01 2.67876598e-01 6.66666667e-03]
 [9.18333760e-01 7.97431632e-02 1.92307692e-03]
 [5.00824688e-01 4.87802763